## AutoCast end-to-end training and evaluation example

This notebook demonstrates end-to-end training of:

- autoencoder
- flow matching
- evaluation


### Example dataaset

We use the `ReactionDiffusion` dataset as an example dataset to illustrate training and evaluation of models. This dataset simulates the advection-diffusion equation in 2D.


In [ ]:
import pickle
from pathlib import Path

from autoemulate.simulations.advection_diffusion import AdvectionDiffusion
from autoemulate.simulations.reaction_diffusion import ReactionDiffusion

from autocast.data.advection_diffusion import (
    AdvectionDiffusion as AdvectionDiffusionMultichannel,
)
from autocast.data.datamodule import SpatioTemporalDataModule, TheWellDataModule
from autocast.metrics.spatiotemporal import MAE, MSE, RMSE

THE_WELL = False
n_steps_input = 1
n_steps_output = 4
stride = 1
rollout_stride = 4


### Read combined data into datamodule


In [ ]:
if not THE_WELL:
    simulation_name = "reaction_diffusion"
    # simulation_name = "advection_diffusion"
    # simulation_name = "advection_diffusion_multichannel"

    if simulation_name == "advection_diffusion_multichannel":
        # Override to use multichannel version
        Sim = AdvectionDiffusionMultichannel
    if simulation_name == "reaction_diffusion":
        Sim = ReactionDiffusion
    if simulation_name == "advection_diffusion":
        Sim = AdvectionDiffusion

    sim = Sim(return_timeseries=True, log_level="error")

    def generate_split(
        simulator, n_train: int = 200, n_valid: int = 20, n_test: int = 20
    ):
        """Generate training, validation, and test splits from the simulator."""
        train = simulator.forward_samples_spatiotemporal(n_train)
        valid = simulator.forward_samples_spatiotemporal(n_valid)
        test = simulator.forward_samples_spatiotemporal(n_test)
        return {"train": train, "valid": valid, "test": test}

    # Cache file path
    cache_file = Path(f"{simulation_name}_cache.pkl")

    # Load from cache if it exists, otherwise generate and save
    if cache_file.exists():
        print(f"Loading cached simulation data from {cache_file}")
        with open(cache_file, "rb") as f:
            combined_data = pickle.load(f)
            for key in ["data", "constant_scalars", "constant_fields"]:
                combined_data["test"][key] = (
                    combined_data["test"][key][:5]
                    if combined_data["test"][key] is not None
                    else None
                )
    else:
        print("Generating simulation data...")
        combined_data = generate_split(sim)
        print(f"Saving simulation data to {cache_file}")
        with open(cache_file, "wb") as f:
            pickle.dump(combined_data, f)

    datamodule = SpatioTemporalDataModule(
        # data=combined_data,
        data_path="../datasets/reaction_diffusion",
        n_steps_input=n_steps_input,
        n_steps_output=n_steps_output,
        stride=n_steps_output,
        batch_size=16,
    )
    ae_datamodule = SpatioTemporalDataModule(
        # data=combined_data,
        data_path="../datasets/reaction_diffusion",
        n_steps_input=n_steps_input,
        n_steps_output=n_steps_output,
        stride=n_steps_output,
        batch_size=16,
        autoencoder_mode=True,
    )
else:
    simulation_name = "turbulent_radiative_layer_2D"
    datamodule = TheWellDataModule(
        well_base_path="../../autocast/datasets/",
        well_dataset_name=simulation_name,
        n_steps_input=n_steps_input,
        n_steps_output=n_steps_output,
        min_dt_stride=1,
        use_normalization=True,
    )


### Set-up logging


In [ ]:
from autocast.logging import maybe_watch_model
from autocast.logging.wandb import create_notebook_logger

logger, watch = create_notebook_logger(
    project="autocast-notebooks",
    name=f"00_e2e_{simulation_name}",
    tags=["notebook", simulation_name],
    enabled=False,
)

In [ ]:
batch = next(iter(datamodule.train_dataloader()))
n_channels = batch.input_fields.shape[-1]
w, h = batch.input_fields.shape[2:4]

In [ ]:
# Compress by a factor of 4, with the spatial autoencoder reducing to 16x16 patches with
# n_latent channels
compression = 4
latent_width, latent_height, latent_stride = 16, 16, 2
n_latent = (w * h * n_channels // (latent_width * latent_height)) // compression

print(f"n_latent channels equals {n_latent} for a compression factor of {compression}")

In [ ]:
64 * 2 / 2

In [ ]:

# Train autoencoder
from autocast.decoders.dc import DCDecoder
from autocast.encoders.dc import DCEncoder
from autocast.models.ae import AE

encoder = DCEncoder(
    in_channels=n_channels,
    out_channels=n_latent,
    # hid_channels=[32, 64],
    # hid_blocks=[2, 2],
    # e.g. for extra factor of 2 downsampling
    hid_channels=[32, 64, 128],
    hid_blocks=[2, 2, 2],
    kernel_size=3,
    stride=2,
    spatial=2,
    pixel_shuffle=False,
    periodic=False,
    dropout=None,
)
decoder = DCDecoder(
    in_channels=n_latent,
    out_channels=n_channels,
    # hid_channels=[64, 32],
    # hid_blocks=[2, 2],
    # e.g. for extra factor of 2 downsampling
    hid_channels=[128, 64, 32],
    hid_blocks=[2, 2, 2],
    kernel_size=3,
    stride=2,
    spatial=2,
    pixel_shuffle=False,
    periodic=False,
    dropout=None,
)

ae = AE(encoder=encoder, decoder=decoder, learning_rate=5e-4)

In [ ]:
encoded = ae.encode(next(iter(ae_datamodule.train_dataloader())))


In [ ]:
print("Encoded shape is:", tuple(encoded.shape))

In [ ]:
import lightning as L

device = "mps"  # "cpu"
trainer = L.Trainer(max_epochs=10, accelerator=device, logger=logger)
trainer.fit(ae, ae_datamodule.train_dataloader(), ae_datamodule.val_dataloader())
trainer.save_checkpoint(f"./{simulation_name}_ae_model.ckpt")

In [ ]:
import matplotlib.pyplot as plt

device = "cpu"
num_examples = 2
for idx, batch in enumerate(ae_datamodule.test_dataloader()):
    inputs = batch.input_fields.to(device)
    outputs, latents = ae.forward_with_latent(batch)
    print("Input shape:", inputs.shape)
    print("Output shape:", outputs.shape)
    print("Latent shape:", latents.shape)
    input_frame = inputs[0, 0, :, :, 1].detach().cpu().numpy()
    output_frame = outputs[0, 0, :, :, 1].detach().cpu().numpy()
    latent_frame = latents[0, 0, :, :, 0].detach().cpu().numpy()
    fig, axs = plt.subplots(1, 4, figsize=(10, 4))
    axs[0].imshow(input_frame, cmap="viridis")
    axs[0].set_title("Input")
    axs[1].imshow(output_frame, cmap="viridis")
    axs[1].set_title("Reconstruction")
    axs[2].imshow(output_frame - input_frame, cmap="viridis")
    axs[2].set_title("Difference")
    axs[3].imshow(latent_frame, cmap="viridis")
    axs[3].set_title("Latent")
    plt.tight_layout()
    plt.show()

    if idx + 1 >= num_examples:
        break


### Example shape and batch


In [ ]:
datamodule.train_dataset[0].input_fields.shape

In [ ]:
batch = next(iter(datamodule.train_dataloader()))

batch.input_fields.shape

In [ ]:
from azula.noise import VPSchedule

from autocast.models.encoder_processor_decoder import EncoderProcessorDecoder
from autocast.nn.unet import TemporalUNetBackbone
from autocast.processors.flow_matching import FlowMatchingProcessor

batch = next(iter(datamodule.train_dataloader()))
n_channels = batch.input_fields.shape[-1]

processor_name = "flow_matching"  # set to "diffusion" to compare
# processor_name = "diffusion"  # set to "flow_matching" to compare

backbone = TemporalUNetBackbone(
    # in_channels=n_channels * n_steps_output,
    # out_channels=n_channels * n_steps_output,
    # cond_channels=n_channels * n_steps_input,
    in_channels=n_latent * n_steps_output,
    out_channels=n_latent * n_steps_output,
    cond_channels=n_latent * n_steps_input,
    mod_features=200,
    hid_channels=(32, 64, 128),
    hid_blocks=(2, 2, 2),
    spatial=2,
    periodic=False,
)

if processor_name == "flow_matching":
    processor = FlowMatchingProcessor(
        backbone=backbone,
        schedule=VPSchedule(),  # accepted for API parity, not used internally
        n_steps_output=n_steps_output,
        # n_channels_out=n_channels,
        n_channels_out=n_latent,
        stride=stride,
        flow_ode_steps=4,
    )
else:
    from autocast.processors.diffusion import DiffusionProcessor

    processor = DiffusionProcessor(
        backbone=backbone,
        schedule=VPSchedule(),
        n_steps_output=n_steps_output,
        # n_channels_out=n_channels,
        n_channels_out=n_latent,
    )

model = EncoderProcessorDecoder(
    encoder_decoder=ae,
    processor=processor,
    train_processor_only=True,
    learning_rate=5e-4,
    test_metrics=[MSE(), MAE(), RMSE()],
)
maybe_watch_model(logger, model, watch)


### Run trainer


In [ ]:
import lightning as L

device = "mps"  # "cpu"
trainer = L.Trainer(max_epochs=10, accelerator=device, logger=logger)
trainer.fit(model, datamodule.train_dataloader(), datamodule.val_dataloader())
trainer.save_checkpoint(f"./{simulation_name}_{processor_name}_model.ckpt")

### Run the evaluation


In [ ]:
trainer.test(model, datamodule.test_dataloader())

### Example rollout


In [ ]:
# A single element is the full trajectory
batch = next(iter(datamodule.rollout_test_dataloader()))

In [ ]:
# First n_steps_input are inputs
print(batch.input_fields.shape)
# Remaining n_steps_output are outputs
print(batch.output_fields.shape)

In [ ]:
# Run rollout on one trajectory
preds, trues = model.rollout(
    batch, stride=rollout_stride, max_rollout_steps=20, free_running_only=True
)

print(preds.shape)
assert trues is not None
print(trues.shape)


In [ ]:
from autocast.metrics.spatiotemporal import MSE

assert trues is not None
assert preds.shape == trues.shape
mse = MSE()
mse_error_spatial = mse(preds, trues)
mse_error = mse(preds, trues)
print("MSE spatial has shape (B,T,C):", mse_error_spatial.shape)
print("MSE overall is a single scalar:", mse_error.shape)

In [ ]:
from IPython.display import HTML

from autocast.utils import plot_spatiotemporal_video

batch_idx = 0
if simulation_name == "advection_diffusion_multichannel":
    channel_names = ["vorticity", "velocity_x", "velocity_y", "streamfunction"]
elif simulation_name == "advection_diffusion":
    channel_names = ["vorticity"]
elif simulation_name == "reaction_diffusion":
    channel_names = ["U", "V"]
else:
    channel_names = None

anim = plot_spatiotemporal_video(
    pred=preds,
    true=trues,
    batch_idx=batch_idx,
    save_path=f"{simulation_name}_{batch_idx:02d}.mp4",
    colorbar_mode="column",
    channel_names=channel_names,
)
HTML(anim.to_jshtml())